# WFSS框提取示例

这个笔记本演示了如何在宽场无缝光谱观测（WFSS）中使用广义世界坐标系统（gWCS）来确定源的位置和波长。它展示了如何根据对应成像观测中的源位置计算WFSS观测中的源位置，以及如何计算沿着物体轨迹的特定像素的波长。

接下来，它展示了如何使用gWCS进行光谱的盒提取，并将一维光谱转换为物理单位。

在这个例子中，我们使用JWST项目01076的曝光数据。我们希望处理具有完整gWCS信息的文件，并且已经应用了平场校正。我们还需要运行管道的光通量校准步骤，以便在WFSS文件的头部（在S_PHOTOM头关键字中）填充光度参考文件的名称。这个参考文件将在下面的提取过程中使用。光度步骤不会改变WFSS曝光中的科学数据值，因为文件中的观察模式（OBS_MODE头关键字）设置为NRC_WFSS。

为了实现这一点，必须对数据运行管道的<b>assign_wcs</b>、<b>flat field</b>和<b>photom</b>步骤。通常，这意味着我们可以简单地从MAST下载*_cal.fits文件，这对于本笔记本中使用的成像模式数据是正确的。然而，正如我们下面所示，我们希望将成像模式的平场应用于WFSS数据。这意味着我们必须下载*_rate.fits文件，并手动对数据运行这些管道步骤。为了保持一致，我们对成像模式数据也执行相同的操作。

JWST探测器在其平场中几乎没有波长依赖性，正如通常对HST WFSS数据所做的那样，在这个例子中，我们让管道将直接交叉滤光片的平场应用于所有成像和WFSS观测。我们不使用特定于WFSS的平场。

一旦数据经过适当的校准，笔记本使用<b>grismconf</b>包在成像和WFSS数据中的源位置之间进行转换，并计算与WFSS数据中给定位置相关的波长。<b>grismconf</b>还使用光度参考文件中的光通量校准曲线，将数据从$DN/sec$单位转换为$F_{lambda}$单位（$erg / sec / cm^2 / \overset{\circ}{A}$）。<b>grismconf</b>将从校准参考数据系统（CRDS）获取所需的NIRCam WFSS配置文件。请注意，必须对数据运行光度步骤，以便获得适当的CRDS灵敏度文件的名称。

注意：在这个阶段，重要的部分不是WCS的绝对准确性。相反，我们依赖于成像和WFSS观测之间的准确自洽性。

作者：N. Pirzkal <br>

创建日期：2024年9月24日

## 目录

1. [包导入](#Package-Imports)

2. [定义函数和参数](#Define-Functions-and-Parameters)

3. [下载数据](#Download-Data)

4. [运行管道步骤](#Run-Pipeline-Steps)

5. [WFSS信息的基本计算](#Basic-Computation-of-WFSS-Information)

      * [计算光的分散位置](#Compute-where-light-gets-dispersed-to)

      * [计算给定对象的光谱轨迹](#Compute-the-spectral-trace-for-a-given-object)

      * [基本框提取](#Basic-Box-Extraction)

## 包导入

In [ ]:
from copy import deepcopy  # 导入深拷贝模块

import matplotlib.pyplot as plt  # 导入绘图库

import numpy as np  # 导入NumPy库

import os  # 导入操作系统接口库

import requests  # 导入HTTP请求库

from scipy.stats import sigmaclip  # 从SciPy库导入sigma裁剪函数

import grismconf  # 导入grismconf模块

from jwst.assign_wcs import AssignWcsStep  # 从JWST库导入WCS分配步骤
from jwst.flatfield import FlatFieldStep  # 从JWST库导入平场校正步骤
from jwst.photom import PhotomStep  # 从JWST库导入光度测量步骤

## 设置 CRDS 路径和服务器

在运行管道步骤之前，我们需要确保我们的CRDS（Calibrated Reference Data System）环境已配置好。这包括定义一个CRDS缓存目录，用于存放校准管道将使用的参考文件。

如果本地CRDS缓存的根目录尚未设置，它将会在主目录中创建。

In [ ]:
# 检查本地 CRDS 缓存目录是否已设置。

# 如果没有设置，则将其设置为用户主目录
if (os.getenv('CRDS_PATH') is None):
    os.environ['CRDS_PATH'] = os.path.join(os.path.expanduser('~'), 'crds')  # 设置 CRDS_PATH 环境变量

# 检查 CRDS 服务器 URL 是否已设置。如果没有设置，则进行设置。
if (os.getenv('CRDS_SERVER_URL') is None):
    os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'  # 设置 CRDS_SERVER_URL 环境变量

# 输出当前使用的 CRDS 路径和上下文
print('CRDS local filepath:', os.environ['CRDS_PATH'])  # 打印本地 CRDS 文件路径
print('CRDS file server:', os.environ['CRDS_SERVER_URL'])  # 打印 CRDS 文件服务器 URL

# 在设置所需的环境变量后导入 crds
from crds import client  # 导入 crds 客户端

# 检查当前 CRDS 服务器是否与环境变量中的 URL 匹配
if client.get_crds_server() != os.environ['CRDS_SERVER_URL']:
    client.set_crds_server('https://jwst-crds.stsci.edu')  # 如果不匹配，则设置 CRDS 服务器

## 定义函数和参数

定义一个函数，通过MAST API将指定文件下载到当前目录。该函数包含身份验证逻辑，但本笔记本中的示例使用公共数据，因此不需要MAST API令牌。

In [ ]:
def get_jwst_file(name, mast_api_token=None, overwrite=False):
    """Retrieve a JWST data file from MAST archive.
    
    Parameters
    ----------
    name : str
        Name of the file to download from MAST
        
    mast_api_token : str
        MAST API token. Required only for proprietary data
        
    overwrite : bool
        If True and the requested file already exists locally, the file will not be downloaded. IF False,
        the file will be downloaded
    """

    # 如果文件已经存在于本地，除非用户设置了overwrite参数，否则不重新下载
    if os.path.isfile(name):
        # 如果overwrite为False，打印信息并跳过下载
        if not overwrite:
            print(f'{name} already exists locally. Skipping download.')
            return
        else:
            # 如果overwrite为True，打印信息并重新下载
            print(f'{name} exists locally. Re-downloading.')

    # MAST下载文件的API URL
    mast_url = "https://mast.stsci.edu/api/v0.1/Download/file"
    # 设置请求参数，包含要下载的文件URI
    params = dict(uri=f"mast:JWST/product/{name}")

    # 如果提供了API token，设置请求头
    if mast_api_token:
        headers = dict(Authorization=f"token {mast_api_token}")
    else:
        headers = {}

    # 发送GET请求以下载文件
    r = requests.get(mast_url, params=params, headers=headers, stream=True)
    # 检查请求是否成功
    r.raise_for_status()

    # 以二进制写入模式打开文件并写入下载的内容
    with open(name, "wb") as fobj:
        # 分块写入文件，块大小为1024000字节
        for chunk in r.iter_content(chunk_size=1024000):
            fobj.write(chunk)

定义一个函数，该函数将对输入的速率文件运行 `assign_wcs` 和平场校正（flat fielding）。

In [ ]:
def run_pipeline_steps(filename):
    """运行 assign_wcs、flat field 和 photom 校准步骤在给定文件上。

    如果文件包含 WFSS 数据，则欺骗管道使用成像模式的平场参考文件。

    Parameters
    ----------
    filename : str
        要运行步骤的输入文件名

    Returns
    -------
    filename : str
        管道步骤保存的输出文件名

    photom : jwst.datamodels.ImageModel
        包含校准数据的 Datamodel 实例
    """

    # 调用 AssignWcsStep 处理输入文件
    assign_wcs = AssignWcsStep.call(filename)

    # 为了将成像模式的平场参考文件应用于数据，
    # 我们需要通过暂时将光阑值更改为 CLEAR 来欺骗 CRDS
    reset_pupil = False  # 初始化重置光阑的标志

    # 检查光阑是否为 GRISM，如果是，则进行处理
    if 'GRISM' in assign_wcs.meta.instrument.pupil:
        true_pupil = deepcopy(assign_wcs.meta.instrument.pupil)  # 保存原始光阑值
        assign_wcs.meta.instrument.pupil = 'CLEAR'  # 将光阑值设置为 CLEAR
        reset_pupil = True  # 设置重置光阑的标志为 True

    # 运行平场步骤
    flat = FlatFieldStep.call(assign_wcs, save_results=True)

    # 运行 photom 步骤以填充 WFSS 灵敏度的名称
    photom = PhotomStep.call(flat, save_results=True)

    # 在平场处理完成后，将光阑值恢复为原始值
    if reset_pupil:
        photom.meta.instrument.pupil = true_pupil  # 恢复光阑值
        photom.save(photom.meta.filename)  # 保存校准后的数据

    # 返回输出文件的名称以及数据模型
    return photom.meta.filename, photom

## 下载数据

下载一个示例成像模式速率文件和相应的WFSS（宽场光谱仪）模式速率文件，从MAST（微波天文数据中心）。

In [ ]:
# 首先，从MAST下载成像和WFSS文件

imaging_file = "jw01076103001_02102_00001_nrcalong_rate.fits"  # 成像文件的名称

wfss_file = "jw01076103001_02101_00001_nrcalong_rate.fits"  # WFSS文件的名称

get_jwst_file(imaging_file)  # 下载成像文件

get_jwst_file(wfss_file)  # 下载WFSS文件

## 运行管道步骤

在成像文件和WFSS文件上运行assign_wcs、平场（flat field）和光度校准（photom calibration）步骤。

In [ ]:
# 运行 AssignWcsStep、FlatFieldStep 和 PhotomStep 处理成像速率文件

imaging_flat_file, imaging_data = run_pipeline_steps(imaging_file)  # 调用管道步骤处理成像文件

In [ ]:
# 运行 AssignWcsStep、FlatFieldStep 和 PhotomStep 处理 WFSS 数据文件

# 调用 run_pipeline_steps 函数处理 wfss_file，返回平场文件和数据
wfss_flat_file, wfss_data = run_pipeline_steps(wfss_file)

## WFSS信息的基本计算

所有WFSS（宽场光谱扫描，Wide Field Slitless Spectroscopy）计算均在探测器坐标空间中进行。分散轨迹的所有特征，包括相对位置的变化和轨迹的全局形状（例如，曲率、偏移等），均通过一系列简单的方程处理。这在<a href="https://www.stsci.edu/files/live/sites/www/files/home/hst/instrumentation/wfc3/documentation/instrument-science-reports-isrs/_documents/2017/WFC3-2017-01.pdf">ISR WFC3 2017-01</a>中进行了描述：“一种更通用的坐标变换方法用于光栅（grisms）”。

在这里，我们假设一个源位于像素坐标（$x$, $y$）处。对于同一源，分散轨迹上单个像素的坐标记为（$x_g$, $y_g$），因此该分散轨迹元素的相对位置相对于源的位置偏移为（$x_g$-$x$, $y_g$-$y$）像素。坐标（$x$, $y$）、（$x_g$, $y_g$）与光的波长$\lambda$之间的函数关系及其逆关系为：

$$

\begin{align}

\delta x = x_g - x = f_x(x,y;t)\\

\delta y = y_g - y = f_y(x,y;t)\\

\lambda = f_\lambda(x,y;t)

\end{align}

$$

和

$$

\begin{align}

t = f^{-1}_x(x,y;\delta x)\\

t = f^{-1}_y(x,y;\delta y)\\

t = f^{-1}_\lambda(x,y;\lambda)

\end{align}

$$

注意，这些函数是相对于参数 $t$ 进行参数化的。这为校准工作提供了一定的灵活性，因为 $t$ 可以被定义得相对任意。然而，在 NIRCam 的光栅（grisms）情况下，$t$ 被选择为 GRISMR 和 GRISMC 的 $\delta x$ 或 $\delta y$，分别因为这些光栅沿 x 方向和 y 方向分散光线。然而，为了额外的便利，$t$ 参数被归一化为 1，使得 $t = 0$ 和 $t = 1$ 对应于分散光谱的蓝光和红光边缘。

使用上述 6 个方程，可以关联任何组合的 ($x$,$y$)、($x'$,$y'$)、$t$ 和 $\lambda$ 值。上述方程在 GRISMCONF 包中实现为 DISPX()、DISPY()、DISPL()、INVDISPX()、INVDISPY() 和 INVDISPL()。

现在我们将使用 Grismconf 包来检索有关 WFSS 文件的信息。请注意，我们使用的是上述校准步骤的输出文件。

In [ ]:
# 这是从上面的WFSS文件管道调用生成的最终输出文件

wfss_file = "jw01076103001_02101_00001_nrcalong_photomstep.fits"  # 定义WFSS文件的路径

In [ ]:
# 加载一个WFSS配置文件以在下面的示例中使用
C = grismconf.Config(wfss_file)  # 使用grismconf模块中的Config类加载指定的wfss_file配置文件

### 计算光的分散位置

在这里，我们展示如何计算与给定波长对应的轨迹上的点的位置，针对位于给定探测器位置 ($x$, $y$) 的源。对于这些计算，我们只需要 WFSS 文件。相应的成像模式文件并不是必需的。

In [ ]:
x = 1000  # 像素 x 坐标

y = 1000  # 像素 y 坐标

wavelength = 3.5  # 波长，单位为微米

我们想要计算 $\hat x$，即波长为 $\lambda$ 的光子在一个像素中的分散量。我们首先使用 $t$ 和 $\lambda$ 之间的关系，然后使用 $\hat x$ 和 $t$ 之间的关系。这个过程是通过对位于 ($x$, $y$) 的物体使用 INVDISPL() 函数来完成的，顺序为 "+1"：

检查可用的订单

In [ ]:
看起来您提到的 "C.orders" 可能是一个数据结构或数据库表的名称，但没有提供具体的代码或上下文。为了帮助您，我需要更详细的信息或代码片段，以便我可以为您添加中文注释。

请提供相关的代码或数据结构示例，这样我才能为您添加行级中文注释。

To calculate \( t \) for the given position and wavelength, we need more specific information about the context or the equations involved. Typically, in physics, \( t \) could represent time, and its calculation might depend on the wave equation or other relevant formulas.

For example, if we are dealing with a wave described by the equation:

\[
y(x, t) = A \sin(kx - \omega t + \phi)
\]

where:
- \( A \) is the amplitude,
- \( k \) is the wave number,
- \( \omega \) is the angular frequency,
- \( \phi \) is the phase constant,
- \( x \) is the position,
- \( t \) is the time.

The wave number \( k \) is related to the wavelength \( \lambda \) by:

\[
k = \frac{2\pi}{\lambda}
\]

And the angular frequency \( \omega \) is related to the frequency \( f \) by:

\[
\omega = 2\pi f
\]

If you provide the specific position \( x \) and wavelength \( \lambda \), along with any other necessary parameters (like amplitude, frequency, or phase), I can help you calculate \( t \). Please provide those details for a more accurate calculation.

In [ ]:
t = C.INVDISPL("+1", x, y, wavelength)  # 调用C库中的INVDISPL函数，传入参数"+1"、x、y和wavelength，计算t

print("t =", t)  # 打印t的值

我们现在可以使用 DISPX() 计算 $\delta x$ 和 $\delta y$：

In [ ]:
# 计算在给定位置 (x, y) 和时间 t 下，x 方向的位移
𝛿x = C.DISPX("+1", x, y, t)  # 调用 DISPX 函数，计算 x 方向的位移

# 计算在给定位置 (x, y) 和时间 t 下，y 方向的位移
𝛿y = C.DISPY("+1", x, y, t)  # 调用 DISPY 函数，计算 y 方向的位移

# 输出 x 方向的位移
print("𝛿x =", 𝛿x)  # 打印 x 方向的位移值

# 输出 y 方向的位移
print("𝛿y =", 𝛿y)  # 打印 y 方向的位移值

最终的像素坐标为：

In [ ]:
xg = x + 𝛿x  # 计算新的x坐标，xg为原始x坐标加上变化量𝛿x

yg = y + 𝛿y  # 计算新的y坐标，yg为原始y坐标加上变化量𝛿y

print("Trace coordinates:", xg, yg)  # 输出新的坐标

或者，我们可以计算光在光谱轨迹上某一特定位置的近似波长。例如，我们希望计算一个像素的波长，该像素位于坐标 ($x_g$, $y_g$)，而该光源的坐标已知为 ($x$, $y$)。由于这是一个光栅 R 光谱，我们可以使用 $\delta x$ 与 t 和 $\lambda$ 之间的关系。

In [ ]:
# 源的位置坐标为 (1000, 1000)，我们正在查看像素坐标 1558 处的像素

x = 1000  # 源的 x 坐标

y = 1000  # 源的 y 坐标

In [ ]:
t = C.INVDISPX("+1", x, y, xg - x)  # 调用C库中的INVDISPX函数，计算反向显示的X坐标

In [ ]:
# 调用C.DISPL函数，计算给定坐标(x, y)和时间t下的波长
wavelength = C.DISPL("+1", x, y, t)

# 打印波长值，单位为微米
print(f"Wavelength = {wavelength} microns")

在这里，我们看到返回了我们在计算 $x_g$ 和 $y_g$ 时使用的3.5微米波长。

### 计算给定对象的光谱轨迹

我们可以以类似的方式计算出我们期望的某个对象的分散一阶轨迹的位置。我们可以使用一系列 $t$ 值来覆盖整个光谱轨迹（在这种情况下，NIRCam 校准假设 $0<t<1$）。

In [ ]:
import numpy as np  # 导入NumPy库，用于数值计算

x = 1000  # 设置变量x的值为1000

y = 1000  # 设置变量y的值为1000

ts = np.arange(0, 1, 0.01)  # 创建一个从0到1（不包括1），步长为0.01的数组

我们可以计算所有对应的轨迹坐标和波长 (x$_g$, y$_g$, $\lambda$)：

In [ ]:
# 计算在给定位置 (x, y) 和时间 ts 下的 x 坐标的位移
xgs = C.DISPX("+1", x, y, ts) + x  # 调用 DISPX 函数计算 x 方向的位移并加上原始 x 坐标

# 计算在给定位置 (x, y) 和时间 ts 下的 y 坐标的位移
ygs = C.DISPY("+1", x, y, ts) + y  # 调用 DISPY 函数计算 y 方向的位移并加上原始 y 坐标

# 计算在给定位置 (x, y) 和时间 ts 下的波长
waves = C.DISPL("+1", x, y, ts)  # 调用 DISPL 函数计算波长

显示探测器上轨迹位置的图像。

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 2))  # 创建一个1行1列的子图，图形大小为20x2英寸

plt.scatter(xgs, ygs, c=waves)  # 绘制散点图，x轴为xgs，y轴为ygs，点的颜色由waves决定

plt.colorbar(label="Wavelength (μm)")  # 添加颜色条，标签为“波长 (μm)”

plt.grid()  # 显示网格

plt.ylabel(r"y$_g$")  # 设置y轴标签为y_g，使用LaTeX格式

plt.xlabel(r"x$_g$")  # 设置x轴标签为x_g，使用LaTeX格式

plt.title("Trace location across the detector")  # 设置图表标题

### 基本盒子提取

一个非常基本的光谱“提取”可以使用上述列出的WFSS（宽场光谱扫描）转换来完成。在这里，我们展示如何对一个良好隔离的目标进行基本的“盒子”提取，即不受场中其他源的重叠光谱的污染。

一个关键概念是源在分散的WFSS（宽场分光光谱）观测中的虚拟位置。我们无法仅使用WFSS数据精确确定该位置，必须依赖额外的信息。源的位置通常是在该区域的直接图像中测量的。在成像和WFSS数据相对于彼此未进行抖动的情况下，WFSS观测中源的位置就是在未抖动成像中相同源的位置。

如果进行了抖动，可以依赖WCS（世界坐标系统）来估计在与WFSS观测相同位置和方向下观察到的成像数据中源的位置。这是通过计算在可用成像数据中源的观测RA（赤经）和Dec（赤纬），然后使用来自抖动WFSS观测的WCS将这些新计算的天体坐标转换回探测器的($x$, $y$)位置来完成的。

#### 检查数据中的源

查看成像数据中的源

In [ ]:
xd, yd = 1562, 696  # 定义变量xd和yd，分别赋值为1562和696

In [ ]:
# 显示图像数据的一个区域
plt.imshow(imaging_data.data[yd - 20:yd + 20, xd - 20:xd + 20], origin="lower", vmin=0, vmax=10)  # 从图像数据中提取中心点周围的40x40像素区域，并设置显示参数

plt.xlabel('Column Number')  # 设置x轴标签为“列号”

plt.ylabel('Row Number')  # 设置y轴标签为“行号”

#### 计算轨迹位置

现在获取成像和WFSS文件中的WCS（世界坐标系统），以便我们可以执行坐标转换并计算该源在WFSS观测中的轨迹位置。

In [ ]:
# 获取从探测器坐标系到世界坐标系的转换
imaging_to_world = imaging_data.meta.wcs.get_transform('detector', 'world')

In [ ]:
# 获取WFSS数据的元数据中的WCS（世界坐标系统）信息，并将其转换为检测器坐标系
wfss_to_pix = wfss_data.meta.wcs.get_transform('world', 'detector')

将成像模式数据中的源的 (x, y) 位置转换为 RA（赤经）和 Dec（赤纬），使用成像模式的 WCS（世界坐标系统）。

In [ ]:
# 将图像坐标 (xd, yd) 转换为世界坐标 (ra, dec)
ra, dec = imaging_to_world(xd, yd)

# 打印转换后的世界坐标
print(ra, dec)

现在将RA（赤经）和Dec（赤纬）转换为WFSS（宽场光谱扫描）数据中探测器上的位置，使用WFSS文件中的WCS（世界坐标系统）。请注意，对于此转换，波长和阶数是必需的输入，但它们实际上并不影响计算。在下面的单元格中，我们将使用波长3.56微米和阶数1，但您会发现更改这些值并不会改变最终的x、y值。

In [ ]:
# 调用 wfss_to_pix 函数，将天体的 RA 和 DEC 转换为像素坐标和波长
x, y, wav, ord = wfss_to_pix(ra, dec, 3.56, 1)  # ra: 右升角, dec: Declination, 3.56: 波长, 1: 订单

# 输出转换后的像素坐标和波长
x, y, wav, ord  # x: x坐标, y: y坐标, wav: 波长, ord: 订单

计算该源光谱的近似边界框。我们利用校准定义光谱在色散方向上t=0和t=1处的边界。

In [ ]:
# 首先计算从名义的 x, y 位置到盒子的左边和右边的距离
# （即 t = 0 和 t = 1 的位置）

ts = np.array([0, 1])  # 创建一个包含 t = 0 和 t = 1 的数组

dxs = C.DISPX("+1", x, y, ts)  # 计算在 x 方向上的位移

dys = C.DISPY("+1", x, y, ts)  # 计算在 y 方向上的位移

dxs, dys  # 返回 x 和 y 方向上的位移

In [ ]:
# 使用上面的距离计算框的左边和右边的 x 位置

x_min = int(x + dxs[0])  # 计算框的左边界 x 位置

x_max = int(x + dxs[1])  # 计算框的右边界 x 位置

# 设置框的高度为 50 像素（在名义位置上方和下方各 25 像素）

y_min = int(y + dys[1] - 25)  # 计算框的下边界 y 位置

y_max = int(y + dys[1] + 25)  # 计算框的上边界 y 位置

x_min, x_max  # 返回框的左边界和右边界的 x 位置

显示要提取的框的图像。

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 5))  # 创建一个1行1列的子图，图像大小为20x5英寸

ax.imshow(wfss_data.data[y_min:y_max, x_min:x_max], origin="lower", vmin=0.25, vmax=1)  # 显示指定区域的图像数据，设置原点在左下角，数据值范围为0.25到1

ax.plot(x + dxs - x_min, y + dys - y_min)  # 绘制经过偏移后的坐标点

ax.set_xlabel('Column Number')  # 设置x轴标签为“列号”

ax.set_ylabel('Row Number')  # 设置y轴标签为“行号”

最简单的提取可以在以下假设下进行：光谱轨迹基本上是线性的，并且照射到一个像素上的光的波长仅是色散方向的函数。因此，我们可以使用近似方法，认为一列中的所有像素对应于相同的波长。这仅适用于几乎平坦的色散，并且在过去常常被假设。这还假设在光谱覆盖的区域内，色散的场依赖性是很小的。

#### 估计背景

在这个最简单的案例中，我们估计并从数据中减去一个简单的背景值，该值被认为是包含我们光谱的区域中的中位数值。这个非常简单的步骤说明了简单框提取的局限性，因为任何这样的估计都会受到场中其他源的影响，或者受到分散背景结构中任何不平坦性的偏倚。

In [ ]:
# 我们想要忽略所有NaN像素，因此定位所有非NaN像素
ok = np.isfinite(wfss_data.data[y_min:y_max, x_min:x_max])  # 检查wfss_data数据中指定区域的每个像素是否为有限值

# 对框内的像素进行Sigma裁剪
clipped, low, upp = sigmaclip(wfss_data.data[y_min:y_max, x_min:x_max][ok], 1.5, 1.5)  # 对非NaN像素进行Sigma裁剪，阈值为1.5

# 计算Sigma裁剪后像素的中位数
med_bck = np.nanmedian(clipped)  # 计算裁剪后像素的中位数，忽略NaN值

print("Background level estimate:", med_bck)  # 输出背景水平估计值

In [ ]:
# 绘制数据的直方图，并添加一条表示中位数水平的线

plt.hist(np.ravel(wfss_data.data[y_min:y_max, x_min:x_max]), 100, range=(0, 1.5))  # 绘制直方图，数据范围为0到1.5，分为100个区间

plt.axvline(med_bck, color='r')  # 添加一条红色竖线表示中位数

plt.title('Histogram of pixel values in extraction box')  # 设置图表标题

plt.xlabel('Signal Value (DN/sec)')  # 设置x轴标签

plt.ylabel('Occurrences')  # 设置y轴标签

#### 指定波长

计算该物体在整个阵列上的波长。我们在这里使用完整的阵列以简化计算，并计算像素与源之间在 x 方向（色散方向）的距离。

In [ ]:
ys, xs = np.indices(np.shape(wfss_data.data))  # 创建与wfss_data数据形状相同的索引网格，ys为行索引，xs为列索引

将x值转换为与名义源位置的距离

In [ ]:
dxs = xs - x  # 计算数组xs与标量x之间的差值，结果存储在dxs中

我们使用光栅配置（grismconf）关系来计算各处的 $t$ 值，然后再次计算各处的波长。

In [ ]:
ts = C.INVDISPX("+1", x, y, dxs)  # 调用INVDISPX函数，计算ts，参数包括"+1"、x、y和dxs

lams = C.DISPL("+1", x, y, ts)     # 调用DISPL函数，计算lams，参数包括"+1"、x、y和ts

在探测器上显示与光谱相同区域的波长数组，我们可以看到现在我们为2D WFSS数据的每个元素都有了波长估计。

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 3))  # 创建一个1行1列的子图，设置图形大小为20x3

tt = ax.imshow(lams[y_min:y_max, x_min:x_max], origin="lower", vmin=2, vmax=4)  # 显示指定范围的波长数据，设置原点在下方，vmin和vmax用于设置颜色映射的范围

cbar = plt.colorbar(tt).ax.set_ylabel('Wavelength (microns)', rotation=270, fontsize=12, labelpad=25)  # 添加颜色条并设置其标签，旋转270度，字体大小为12，标签与颜色条的间距为25

plt.title('Wavelength array covering all pixels of extraction box')  # 设置图形标题

plt.xlabel('Column Number')  # 设置x轴标签为“列号”

plt.ylabel('Row Number')  # 设置y轴标签为“行号”

#### Box extraction

盒子提取

从上述数据创建一维光谱（1D spectrum）在其最简单的形式上，就是沿着交叉色散方向（y方向）进行压缩。 一维光谱将受到坏像素（bad pixels）、任何光谱污染（spectra contamination）以及背景减除量（background subtracted）中的任何误差的影响。 随着盒子大小（box size）的增加，这些影响变得越来越重要。我们还从每一列中减去中位数背景（median background）。

In [ ]:
# 计算指定区域内的信号总和，减去背景值
cs = np.nansum(wfss_data.data[y_min:y_max, x_min:x_max] - med_bck, axis=0)

# 绘制信号总和的曲线
plt.plot(cs)

# 设置y轴的范围
plt.ylim(-5, 20)

# 显示网格
plt.grid()

# 设置x轴标签
plt.xlabel('Column Number')

# 设置y轴标签
plt.ylabel('Signal (DN/sec)')

# 设置图表标题
plt.title('Extracted 1D spectrum versus detector column')

#### 波长校准

在这种情况下，我们可以在交叉色散方向（y方向）上对波长数组进行平均，以获得一个适合我们刚刚框提取的光谱的单一波长向量。将由此创建的1D计数向量与波长向量绘制在一起，结果是一个波长校准的1D光谱。

In [ ]:
ws = np.nanmean(lams[y_min:y_max, x_min:x_max], axis=0)  # 计算指定区域内的波长平均值，忽略NaN值

plt.plot(ws, cs)  # 绘制波长与信号的关系图

plt.ylim(-5, 20)  # 设置y轴的范围为-5到20

plt.grid()  # 显示网格

plt.xlabel('Wavelength (microns)')  # 设置x轴标签为“波长（微米）”

plt.ylabel('Signal (DN/sec)')  # 设置y轴标签为“信号（DN/秒）”

plt.title('Extracted 1D spectrum versus wavelength')  # 设置图表标题为“提取的1D光谱与波长的关系”

#### 转换为物理单位

在这里，我们使用包含在 grismconf 中的信息，将我们的 1D 光谱（单位为 DN/sec，按 bin 或像素计算，因为数据没有重新采样）转换为单位为 $erg / sec / cm^2 / \overset{\circ}{A}$。grismconf 中的光栅逆灵敏度曲线单位为 DN/sec（每像素）每 $erg / sec / cm^2 / \overset{\circ}{A}$。由于灵敏度是按埃（Angstrom）定义的，而我们的 1D 光谱是按像素提取的，因此在应用灵敏度时需要考虑这一点。在大多数 WFSS 数据中，波长单位的像素大小变化较小，但我们可以通过计算每个像素在波长单位（埃）中的大小来轻松处理这一点，如下所示。

In [ ]:
# 绘制逆灵敏度曲线

tws = np.arange(2.9, 4.2, 0.01)  # 创建一个从2.9到4.2的数组，步长为0.01

plt.plot(tws, C.SENS["+1"](tws))  # 绘制逆灵敏度曲线

plt.grid()  # 添加网格线

plt.xlabel("Wavelength (micron)")  # 设置x轴标签为“波长（微米）”

plt.ylabel("Inverse Sensitivity")  # 设置y轴标签为“逆灵敏度”

plt.title("Inverse sensitivity curve")  # 设置图表标题为“逆灵敏度曲线”

计算我们一维光谱中每个元素的大小，单位为埃（Angstrom）。

In [ ]:
# 10000的值来自NIRCam的色散值，约为每像素10Å

dws = (ws[1:] - ws[:-1]) * 10000  # 计算相邻波长之间的差值并乘以10000以转换单位

通过将一维计数除以灵敏度和每个波段的大小（以埃为单位），应用灵敏度曲线，以生成单位为 $erg / sec / cm^2 / \overset{\circ}{A}$ 的通量值。

In [ ]:
fs = cs[1:] / C.SENS["+1"](ws[1:]) / dws  # 计算频谱 fs，使用 cs 数组的切片，调用 C.SENS 函数并进行归一化

绘制结果的通量数组与我们的波长尺度相比，显示了完全校准的1D光谱。将DN/秒每像素转换为$erg / sec / cm^2 / \overset{\circ}{A}$时的一个注意事项是，众所周知的边缘效应，这种效应是由于将接近零或为零的计数值除以灵敏度而引起的。由于这种方法没有考虑到物体在色散方向上的足迹导致的光谱展宽，因此对于扩展源，这种效应更为严重。只有通过更前向建模的提取方法才能减轻这种效应。

In [ ]:
plt.plot(ws[1:], fs)  # 绘制波长ws（从第二个元素开始）与信号fs的关系图

plt.xlabel("Wavelength (micron)")  # 设置x轴标签为“波长（微米）”

plt.ylabel("Signal ($erg/s/cm^2/A$)")  # 设置y轴标签为“信号（$erg/s/cm^2/A$）”

plt.title("Extracted 1D spectrum in $F_{lambda}$ units")  # 设置图表标题为“提取的1D光谱（$F_{lambda}$单位）”

集中在光谱边缘的区域可以揭示我们源的光谱。这种简单的盒子提取方法的众多缺点之一是坏像素和任何残留宇宙射线的影响。这些因素可能在光谱中造成显著的峰值，可能会被误认为是发射线。

In [ ]:
plt.plot(ws[1:], fs)  # 绘制波长(ws)与信号(fs)的关系图，ws从第二个元素开始

plt.xlim(3.1, 4.)  # 设置x轴范围为3.1到4.0微米

plt.ylim(0, 0.5e-16)  # 设置y轴范围为0到0.5e-16 erg/s/cm²/Å

plt.xlabel("Wavelength (micron)")  # 设置x轴标签为“波长（微米）”

plt.ylabel("Signal ($erg/s/cm^2/A$)")  # 设置y轴标签为“信号（erg/s/cm²/Å）”

plt.title("Extracted 1D spectrum in $F_{lambda}$ units")  # 设置图表标题为“提取的1D光谱（$F_{lambda}$单位）”